In [1]:
import os
import sys
import pandas
import numpy
import shutil
import cProfile, pstats, io

%load_ext cython

In [2]:
from algorithm_tester.tester_logic import run_algorithms_for_file
from algorithm_tester.tester_dataclasses import AlgTesterContext
from algorithm_tester.helpers import create_path

In [3]:
def profile(fnc):
    
    """A decorator that uses cProfile to profile a function"""
    
    def inner(*args, **kwargs):
        
        pr = cProfile.Profile()
        pr.enable()
        retval = fnc(*args, **kwargs)
        pr.disable()
        s = io.StringIO()
        sortby = 'cumulative'
        ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
        ps.print_stats()
        print(s.getvalue())
        return retval

    return inner

In [4]:
@profile
def profileRunAlgorithms(context, input_file):
    run_algorithms_for_file(context, input_file)

out_dir: str = ".tmp_out_dir"
context: AlgTesterContext = AlgTesterContext(
    algorithms=["SA"], parser="KnapsackParser", communicators=[],
    check_time=True, time_retries=1, max_num=None,
    extra_options={"init_temperature":2500, "min_temperature":1, "cooling":0.995, "cycles":50},
    input_dir="../data/NK", output_dir=out_dir
    )
    
create_path(out_dir)
with open("../data/Profile/Profile_40_inst.dat") as input_file:
    profileRunAlgorithms(context, input_file)
    
shutil.rmtree(out_dir)

Running output for: Profile_40_inst_SA_sol.dat. Started 19:59:32 28.12.
         266 function calls in 0.064 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.064    0.064 <ipython-input-4-b22bf7c8de01>:1(profileRunAlgorithms)
        1    0.001    0.001    0.064    0.064 /Users/petr/Documents/Projects/Python/AlgorithmTester/algorithm_tester/tester_logic.py:67(run_algorithms_for_file)
        2    0.000    0.000    0.062    0.031 /Users/petr/Documents/Projects/Python/AlgorithmTester/algorithm_tester/tester_logic.py:34(get_instance_file_results)
        1    0.000    0.000    0.061    0.061 /usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/Versions/3.7/lib/python3.7/timeit.py:162(timeit)
        1    0.000    0.000    0.061    0.061 <timeit-src>:2(inner)
        1    0.000    0.000    0.061    0.061 /Users/petr/Documents/Projects/Python/AlgorithmTester/algorithm_tester/tester_logic.p

In [25]:
%%cython

import time
import random
from libc.math cimport exp
from libc.stdlib cimport rand, srand, RAND_MAX

cpdef void set_seed(int seed):
    srand(seed)
    
cpdef float random_float():
    return rand()/RAND_MAX

cpdef int random_int(int low = 0, int height = 2):
    return rand() % height + low

cpdef void file_test(str path, int size):
    cdef str value
    with open(path, 'w') as fout:
        for i in range(size):
            value = f'{i};25;45;12\n'
            fout.write(value)

In [27]:
@profile
def test(path: str, size: int = 100000):
    file_test(path, size)
    
test("test_file.csv")

         6 function calls in 0.038 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.038    0.038 <ipython-input-27-d676c1ad7d67>:1(test)
        1    0.038    0.038    0.038    0.038 {built-in method _cython_magic_a70ea1bd4dbf5eb91d5809c58b34405d.file_test}
        1    0.000    0.000    0.000    0.000 /usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/Versions/3.7/lib/python3.7/_bootlocale.py:33(getpreferredencoding)
        1    0.000    0.000    0.000    0.000 {built-in method _locale.nl_langinfo}
        1    0.000    0.000    0.000    0.000 /usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/Versions/3.7/lib/python3.7/codecs.py:186(__init__)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}



